In [4]:
import pandas as pd
import numpy as np
import pybaseball


This block will download all of the raw data from 2017 through 2021


Some thoughts:
    I have two options:
        1) Download the data and cut it down/clean it to what it I want,
        2) Download the data, save that, then clean it and save that
        
        Option 1 is more memory efficient but option 1 is more time but 
        option 2 is more time efficient if/when I make mistakes
        
        So for now I'm going with option two and if that is super fast then I will
        delete that result and combine them into one script. Okay here goes

In [6]:
def get_data_from_mlb(start_year, end_year = None):
    
    if end_year == None:
        end_year = start_year + 1
    
    dfs = [] #https://github.com/jldbc/pybaseball/blob/master/EXAMPLES/imputed_derivation.ipynb
    for year in range(start_year, end_year + 1):
        print(f"Starting year {year}")
        dfs.append(pybaseball.statcast(start_dt=f'{year}-03-21', end_dt=f'{year}-12-01',verbose=False))
        #I'm overshooting the start and end here to make sure I don't miss anything
        
    # https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
    print("Info succesfully downloaded from statcast")
    return pd.concat(dfs) 
    

In [ ]:
if __name__ == '__main__':

    result = get_data_from_mlb(2017, 2021)
    
    print("Writing raw data to csv")
    result.to_csv("raw_data_from_statcast.csv", index = False)
    
    
    
    key_points = result[[ 
                            "game_date",
                            "description",
                            "home_team",
                            "away_team",
                            "game_year",
                            "plate_x",
                            "plate_z",
                            "sz_top",
                            "sz_bot",
                            "game_pk"
    ]]
    
    print("Writing key_points data to csv")
    key_points.to_csv("key_points.csv", index = False)
    
    

key_points.csv has all of the pitch information I need to work with. 

Now I need to group them by game, and rate the ump's performance for that game

In [ ]:
class Rate_Game:
    """
    Inputs:
        game_df: 
            pandas df of all of the pitches 
            In most cases this will be one game, but I guess I could use this to get overall averages too
        game_pk:
            ID of the game
    Output:
        A pandas series that will have the following information:
        game_pk, game_date, ump accuracy, ump consistency
    """
    
    def grade_game(game_df, game_pk):
    sz_L = -.8308
    sz_R = .8308
    
    #https://stackoverflow.com/questions/30631841/pandas-how-do-i-assign-values-based-on-multiple-conditions-for-existing-columns
    game_df["accurate"] = np.where(
        (game_df["plate_x"] > sz_L) & \
        (game_df["plate_x"] < sz_R) & \
        (game_df["plate_z"] > game_df["sz_bot"]) & \
        (game_df["plate_z"] < game_df["sz_top"]),
        True, False
    ) 
    
    #Now we need to figure out consistency
    #Then build & return the series
    
    print(game_df)
    exit()


In [ ]:
key_points = pd.read_csv("key_points.csv")
game_groups = key_points.groupby(['game_pk'])